In [1]:
from azure.devops.connection import Connection
from msrest.authentication import BasicAuthentication
import pprint
import os
from bs4 import BeautifulSoup
import requests
import json
import jsonschema
from jsonschema import validate
import re
from datetime import datetime

### Git Client connection to Azure DevOps

The code below uses connects to your Organization and uses a Personal Access Token (PAT) to authenticate to Azure DevOps  

You will need to provide:
1. the url to your organization 
2. a PAT against the organization that is listed in the URL


In [2]:
# Get a client (the "core" client provides access to projects, teams, etc)
def get_clients_projects():

    # Fill in with your personal access token and org URL
    personal_access_token = os.getenv("gitpat")
    organization_url = os.getenv("org_url")

    # Create a connection to the org
    credentials = BasicAuthentication('', personal_access_token)
    connection = Connection(base_url=organization_url, creds=credentials)    
    
    core_client = connection.clients.get_core_client()
    git_client = connection.clients.get_git_client()

    # Get the first page of projects
    get_projects_response = core_client.get_projects()
    return core_client, git_client, get_projects_response

### Simple JSON validation

This is not 100% accurrate but is sufficient for most use cases in testing whether a string is in fact a valid JSON string.

In [3]:
def json_validator(data):
    try:
        json.loads(data)
        return True
    except ValueError as error:
        print("invalid json: %s" % error)
        return False

### Reading all JSON files in the pipeline folder

The following will read all files in the pipeline folder for every repository in an organization. Further, it will modify only the specified branch in any repository that has a matching name. The ADF Activity type that is is specifically looking for is the "copy" activity. If you need to search on more then a for loop should be created to iterate through all the activities.

In [ ]:
core_client, git_client, get_projects_response = get_clients_projects()
from utils.json_builder import Change
from utils.helper import is_empty, get_all_keys, look_in_string
import json


from base64 import b64encode
from base64 import decode
import requests

from requests.auth import HTTPBasicAuth

branch_to_find = 'test1'

# whole number
retryIntervalInSeconds = 90

# whole number
retry = 52

# string value formatted as Days, Hours, Minutes, Seconds {0.0:00:00}
timeout = "0.0:15:00"

git_commit = dict()

index = 0
while get_projects_response is not None:
    for project in get_projects_response.value:
        repos = git_client.get_repositories(project.id)
        for repo in repos:
            try:
                branches = git_client.get_branches(repository_id=repo.id)
                for branch in branches:
                    if branch.name != branch_to_find:
                        continue

                    commit = git_client.get_commit(repository_id=repo.id, commit_id=branch.commit.commit_id)

                    try:
                        mychange
                        del mychange
                    except NameError:
                        pass

                    mychange = Change(branch_name=branch.name, commit_id=commit.commit_id, comment="update adf pipelines" + datetime.now().strftime("%Y %B, %A %w, %H hours %M minutes"))
                    
                    tree = git_client.get_tree(repository_id=repo.id, sha1=commit.tree_id)

                    for entry in tree.tree_entries:
                        if entry.relative_path == "pipeline":
                            cc = git_client.get_tree(repository_id=repo.id, sha1=entry.object_id)

                            for c in cc.tree_entries:
                                if c.git_object_type == 'blob':
                                    try:

                                        blob = git_client.get_blob(repository_id=repo.id, sha1=c.object_id)

                                        r = requests.get(blob.url, auth=HTTPBasicAuth('',os.getenv("gitpat")))

                                        content = json.dumps(json.loads(r.content))

                                        what_i_found = look_in_string(content,"\"timeout\"",",")
                                        for found in what_i_found:
                                            content = content.replace(found,f'"timeout": \"{timeout}\"')

                                        what_i_found = look_in_string(content,"\"retryIntervalInSeconds\"",",")
                                        for found in what_i_found:
                                            content = content.replace(found,f'"retryIntervalInSeconds": {retryIntervalInSeconds}')

                                        what_i_found = look_in_string(content,"\"retry\"",",")
                                        for found in what_i_found:
                                            content = content.replace(found,f'"retry": {retry}')

                                    except Exception as ex:
                                        print(f'failed to read json file {ex}')

                                    try:
                                        content = json.loads(content)
                                    except Exception as mm:
                                        print(f'error on converting to dict: {mm}')

                                    # make sure to use the indent as the json.loads removes
                                    # all tabs and returns. This formats the json string
                                    # back to the appropriate look for a json string
                                    # in ADF
                                    my_json = json.dumps(content,indent=4)

                                    file_path = f"{entry.relative_path +'/'+ c.relative_path}"
                                    print(f'what is the path of interest {file_path}')

                                    mychange.add_content(path=file_path, content=my_json)
                                    del my_json

                     
                            output = mychange.git_push()
                            # print(f'>>>>{output}')
                            # print(f'Repository name:{repo.name}\nBranch name:{branch.name}\nCommit ID:{commit.commit_id}\nFile Path:{file_path}')

                            git_client.create_push(repository_id=repo.id, push=json.loads(output))
                            print("\t\tJust sent the code to git repo")

                            # delete objects so they can be created clean within the loop
                            print('Deleting file_path')
                            del file_path

                            print('Deleting mychange')
                            del mychange

            except Exception as e:
                try:
                    file_path
                    del file_path
                except NameError:
                    pass

                try:
                    mychange
                    del mychange
                except NameError:
                    pass
                
                print(e)
        index += 1

        #project.
    if get_projects_response.continuation_token is not None and get_projects_response.continuation_token != "":
        # Get the next page of projects
        get_projects_response = core_client.get_projects(continuation_token=get_projects_response.continuation_token)

    else:
        # All projects have been retrieved
        get_projects_response = None
